In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import logging
import sys
import time, datetime
from typing import Dict, Iterable
import os
import gzip
import json

from google.colab import drive

# Mount Google Drive to the current Colab session
drive.mount('/content/drive')

# Verify that your drive is accessible
!ls /content/drive/MyDrive

Mounted at /content/drive
'16 б.gdoc'			  deleteme.zip		   share
'6 фото.     5 лет блондин.JPG'   futuristic-office.jpeg  'Tanya Passport.JPG'
'Colab Notebooks'		  rental_houses.gsheet	  'лечение травами 3.gdoc'


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
def remove_triple_backticks_at_edges(text: str) -> str:
    """
    Removes triple backticks (` ``` `) if they appear at the beginning or the end of the text.

    :param text: The input text as a string.
    :return: A string with triple backticks removed from the edges.
    """
    s1 = "```python\n"
    s2 = "```python"
    s3 = "```"
    for s in [s1, s2, s3]:
        if text.startswith(s):
            text = text[len(s):].lstrip()      # Remove a version of backticks + extra whitespace at the start
    if text.endswith(s3):
        text = text[: -len(s3)].rstrip()       # Remove the backticks + extra whitespace at the end
    return text

In [4]:
def write_jsonl(filename: str, data: Iterable[Dict], append: bool = False):
    """
    Writes an iterable of dictionaries to jsonl
    """
    if append:
        mode = 'ab'
    else:
        mode = 'wb'
    filename = os.path.expanduser(filename)
    if filename.endswith(".gz"):
        with open(filename, mode) as fp:
            with gzip.GzipFile(fileobj=fp, mode='wb') as gzfp:
                for x in data:
                    gzfp.write((json.dumps(x) + "\n").encode('utf-8'))
    else:
        with open(filename, mode) as fp:
            for x in data:
                fp.write((json.dumps(x) + "\n").encode('utf-8'))


def stream_jsonl(filename: str) -> Iterable[Dict]:
    """
    Parses each jsonl line and yields it as a dictionary
    """
    if filename.endswith(".gz"):
        with open(filename, "rb") as gzfp:
            with gzip.open(gzfp, 'rt') as fp:
                for line in fp:
                    if any(not x.isspace() for x in line):
                        yield json.loads(line)
    else:
        with open(filename, "r") as fp:
            for line in fp:
                if any(not x.isspace() for x in line):
                    yield json.loads(line)


def read_problems(evalset_file: str) -> Dict[str, Dict]:
    return {task["task_id"]: task for task in stream_jsonl(evalset_file)}

In [12]:
def is_full_function(input_str: str) -> str:
    '''
    Determine if input string is a Python function
    using simple heuristics (all that's needed)
    '''
    lines = input_str.splitlines()
    for line in lines:
        if line.startswith('def '):
            return True
    return False


def remove_triple_backticks_at_edges(text: str) -> str:
    """
    Removes triple backticks (` ``` `) if they appear at the beginning or the end of the text.

    :param text: The input text as a string.
    :return: A string with triple backticks removed from the edges.
    """
    s1 = "```python\n"
    s2 = "```python"
    s3 = "```"
    for s in [s1, s2, s3]:
        if text.startswith(s):
            text = text[len(s):].lstrip()      # Remove a version of backticks + extra whitespace at the start
    if text.endswith(s3):
        text = text[: -len(s3)].rstrip()       # Remove the backticks + extra whitespace at the end
    return text


def add_import(text: str) -> str:
    ''' Adds import statements stripped by LLM '''
    text = text.strip()
    s = 'from typing import List'
    if 'def ' in text and s not in text:
        text = s + '\n\n' + text
    return text

string = 'def my_func(input):\n    return output'
print(add_import(string))

from typing import List

def my_func(input):
    return output


In [5]:
# read dataset
file = '/content/drive/My Drive/Colab Notebooks/data/LBPP_Test.jsonl.gz'
tasks = read_problems(file)
print(f'Type of tasks: {type(tasks)}\n')

counter = 0
for k,v in tasks.items():
    if counter == 10:
        break
    print(f'task_id: {k}', type(k))
    for k2, v2 in v.items():
        print(f'\n{k2}:\n{v2}')
    print('\n' + '='*100 + '\n')
    counter += 1

Type of tasks: <class 'dict'>

task_id: lbpp/0 <class 'str'>

task_id:
lbpp/0

test:
import numpy as np

ar = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
expected_output = np.array([[1, 2, 3, 2.0, 0.81649658], [4, 5, 6, 5.0, 0.81649658], [7, 8, 9, 8.0, 0.81649658]])
assert np.allclose(add_avg_and_std_cols_numpy(ar), expected_output)
ar = np.array([[1], [1], [2]])
expected_output = np.array([[1, 1, 0.0], [1, 1, 0.0], [2, 2, 0.0]])
assert np.allclose(add_avg_and_std_cols_numpy(ar), expected_output)
ar = np.array([[1, 2, 3, 4, 5]])
expected_output = np.array([[1, 2, 3, 4, 5, 3.0, 1.41421356]])
assert np.allclose(add_avg_and_std_cols_numpy(ar), expected_output)

prompt:
Write a python function `add_avg_and_std_cols_numpy(ar: np.ndarray) -> np.ndarray` that takes a 2D numpy array and returns a 2D numpy array with two additional columns appended to the end. The first column should contain the average of each row, and the second column should contain the standard deviation of each row. You ma

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    "NTQAI/Nxcode-CQ-7B-orpo",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("NTQAI/Nxcode-CQ-7B-orpo")
print(f'Model device: {model.device}')

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/31.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/7.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/6.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/972 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.46M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

Model device: cuda:0


In [7]:
print(f'Model temperature: {model.config.temperature}. Model top_p: {model.config.top_p}')

Model temperature: 1.0. Model top_p: 1.0


In [8]:
def generate_response(prompt):
  messages = [
      {"role": "user", "content": prompt}
  ]
  inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
  outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
  res = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
  return res

In [10]:
task = tasks['lbpp/15']
prompt_prefix = 'You are an expert Python programmer, and here is your task: {} Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:\n'

prompt_prefix = '''1. You are an expert Python programmer, and here is your task: {}.
2. Your output must include only the correct import statements, the function with proper Python code, and and nothing after it. Stop generating immediately after the return statement or final line of the function.
3. You output should be runnable Python code. Do not include phrases like "Solution:" or any other headings, do not show example usage, do not show test cases, do not provide explanatory text.
4. Your output must be with no additional text, no extra symbols, and no code fences (```).
5. Your code must satisfy these tests:

Test cases:

'''
print(prompt_prefix.format(task['prompt']) + task['test'])

1. You are an expert Python programmer, and here is your task: Write a python function "def bonus_calculation(employees: pd.DataFrame,
bonus_categories: Dict[str,float], additional_bonus: float) -> Dict[int,int]" that takes as input
the dataframe "employees", containing the following columns: employee_id, employee_name,
salary, total_in_sales. Additionally, the function takes a dictionary
containing the bonus categories ("Low" if total in sales is less than 10000, "Medium" if
total in sales is between 10000 and 20000 and "High" if total in sales is higher than
20000) with the correspondent bonus percentage that should be applied on each employee's
salary. Lastly, the additional_bonus is a percentage of the base salary (before the first bonus is applied) that should be added on top of the
regular bonus, but only for those who achieved the "High" bonus category. All the percentages
are in the decimal form, e.g. 0.10 for 10%.
Return a dictionary containing employee_id and the calculated b

In [11]:
# to save results
model_nickname = 'nxcode'
time_stamp     = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-2]
file_name      = f'/content/drive/My Drive/Colab Notebooks/logs/{model_nickname}_samples_{time_stamp}.jsonl'
file_name

'/content/drive/My Drive/Colab Notebooks/logs/nxcode_samples_20250120_091920_8866.jsonl'

In [13]:
num_samples_per_task = 1
completions = []
is_full_func = []

for task_id, task_body in tasks.items():
    print(task_id)
    #if task_id == '15':
    #        break
    for i in range(num_samples_per_task):
        start_time  = time.time()
        full_prompt = prompt_prefix.format(task_body['prompt']) + task_body['test']
        try:
            completion  = generate_response( full_prompt )
        except Exception as e:
            completion = f"Error generating a completion:\n{e}"

        completion = remove_triple_backticks_at_edges(completion.strip())
        is_full = is_full_function(completion)
        is_full_func.append(is_full)
        if is_full:
            completion = add_import(completion)
        completions.append( {'task_id': task_id, 'completion': completion} )
        write_jsonl(file_name, completions)
        print('Prompt:\n', full_prompt, '\n', '-'*75, '\n', sep='')
        print('Completion:\n', completion, sep='')
        print(f"\nTime elapsed: {(time.time() - start_time):.4f} seconds\n", '\n', '='*75, '\n', sep='')

lbpp/0


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
n2 = ListNode(4)
n3 = ListNode(1)
n4 = ListNode(1)
n5 = ListNode(2)
n6 = ListNode(3)
n7 = ListNode(5)
n1.next = n2
n2.next = n3
n3.next = n4
n4.next = n5
n5.next = n6
n6.next = n7
new_list = linked_list_fibonacci(n1)
assert new_list == n3
assert new_list.next == n4
assert new_list.next.next == n5
assert new_list.next.next.next == n6
assert new_list.next.next.next.next == n7
assert new_list.next.next.next.next.next == None
n1 = ListNode(1)
n2 = ListNode(4)
n3 = ListNode(1)
n4 = ListNode(2)
n5 = ListNode(3)
n6 = ListNode(5)
n1.next = n2
n2.next = n3
n3.next = n4
n4.next = n5
n5.next = n6
new_list = linked_list_fibonacci(n1)
assert new_list == n1
assert new_list.next == n3
assert new_list.next.next == n4
assert new_list.next.next.next == n5
assert new_list.next.next.next.next == n6
assert new_list.next.next.next.next.next == None
n1 = ListNode(1)
n2 = ListNode(4)
n3 = ListNode(3)
n4 = ListNode(1)
n5 = ListNode(2)
n6 = ListNode(3)
n1.next 

In [14]:
# count how many full funcs were generated vs. only func bodies without func signatures
from collections import Counter
Counter(is_full_func)

Counter({True: 159, False: 3})

In [ ]:
from google.colab import runtime
runtime.unassign()